# Part 2 Design the Low Order Filter Using a Single Kernel in AIE

A finite impulse response (FIR) filter is described by the following equation, where x denotes the input, C denotes the coefficients, y denotes the output, and N denotes the length of the filter.

$$
y_{n} = \sum_{k=0}^{N-1}C_{k}\ast x_{n+k}
$$

Data is sequenced through the filter and at each tap is multiplied by a filter coefficient, or in the symmetric case two data taps are added then multiplied by their common coefficient to reduce the number of multipliers. The multiplicands are summed to generate the output, and the data progresses through taps sequentially on each clock. These familiar structures are readily implemented in programmable logic fabric or in dedicated blocks such as the DSP48 or DSP58. The question is, how does this translate into a Versal™ AIE implementation?

The AIE performs vector processing to handle FIR calculations: data and coefficients are packed into vectors and submitted to the processor’s X and Z registers respectively, the VLIW processor calculates the outputs, and the results are stored in the AIE accumulator. The accumulator is split into lanes, so depending on the data type either 4, 8, or 16 FIR calculations can be performed in parallel (4 lanes are shown for illustration in figure 3). The accumulator data is then adjusted to the desired output data width and output as the FIR output vector. This AIE process can handle both asymmetric and symmetric FIR filters, real and/or complex data and coefficients.

The following figure shows the main steps to be covered in part 2.

```mermaid

graph TD

    A[Design Specification] --> B(Sample Rate)
    A[Design Specification] --> C(Symmetry)
    A[Design Specification] --> D(Data Type)
    A[Design Specification] --> E(Coefficient)
    B(Sample Rate) --> F[Interface Type]
    C(Symmetry) --> F[Interface Type]
    D(Data Type) --> G[Max MACs per Cycle]
    E(Coefficient) --> G[Max MACs per Cycle]
    G[Max MACs per Cycle]  --> H[Lanes and Samples]
    H[Lanes and Samples]  --> I[Vector Data Type and Pointer Position] 
    I[Vector Data Type and Pointer Position] --> K[API calls for I/O]
    F[Interface Type] --> K[API calls for I/O]
    I[Vector Data Type and Pointer Position] --> L[API calls for Computation]
    K(API calls for I/O) --> M(window_readincr_v<8> aie::load_v)
    L(API calls for Computation) --> N(Special Multiplications aie::sliding_mul_sym_xy_ops)

```

Reference:

- [UG1076 Versal ACAP AI Engine Programming Environment User Guide](https://docs.xilinx.com/r/en-US/ug1076-ai-engine-environment/Overview)
- [AM009 Versal ACAP AI Engine Architecture Manual](https://docs.xilinx.com/r/en-US/am009-versal-ai-engine)
- [AI Engine API Documentation](https://www.xilinx.com/htmldocs/xilinx2023_1/aiengine_api/aie_api/doc/index.html)

## Step1: Determine the Design Specifications

Low-pass FIR, sampling at 1 GSPS with a cutoff frequency of 10 Hz, and at least 60 dB of rejection. Input data is 16-bit complex format, and coefficients are 16-bit signed format. Coefficients are symmetric.

| Symmetric | Tap - Points   |  CoeffType   | DataType | Samples |
|--------|---------|----------|--------------|--------|
|   Yes  | 16 | int16  | cint16 | 1,024 |

## Step 2: Find the Maximum Performance for the Given Data Type

With your estimated filter length in mind the process starts with the driving parameter – data type. There are three key points regarding AIE data types:

1. The AIE processing capability depends on the data types involved, both the data and the coefficients.

2. Arbitrary precisions are not supported, so any data types that do not match the data types handled by the AIE must be extended or truncated to match one of the supported types. The supported data widths are 8, 16, and 32 bits.

3. The FIR data is loaded as the X operand of the AIE VLIW processor, and the coefficients are loaded as the Z operand.

With these points in mind, refer to the table “Supported Precision Width of the Vector Datapath” in the latest copy of UG1076. For convenience that table is reproduced here, but you should still verify in the
original document in [AM009](https://docs.xilinx.com/r/en-US/am009-versal-ai-engine/AI-Engine-Architecture).

| X Operand  | Z Operand  | Output        | Number of MACs | 
|------------|------------|---------------|:--------------:|
| 8 real     | 8 real     | 48 real       |       128      |  
| 16 real    | 8 real     | 48 real       |       64       |   
| 16 real    | 16 real    | 48 real       |       32       |   
| 16 real    | 16 complex | 48 complex    |       16       |   
|<strong> 16 complex<strong> | <strong>16 real<strong>    | <strong>48 complex<strong>  | <strong>  16 <strong>  |   
| 16 complex | 16 complex | 48 complex    |        8       |   
| 16 real    | 32 real    | 48/80 real    |       16       |   
| 16 real    | 32 complex | 48/80 complex |        8       |   
| 16 complex | 32 real    | 48/80 complex |        8       |   
| 16 complex | 32 complex | 48/80 complex |        4       |   
| 32 real    | 16 real    | 48/80 real    |       16       |   
| 32 real    | 16 complex | 48/80 complex |        8       |   
| 32 complex | 16 real    | 48/80 complex |        8       |   
| 32 complex | 16 complex | 48/80 complex |        4       |   
| 32 real    | 32 real    | 80 real       |        8       |   
| 32 real    | 32 complex | 80 complex    |        4       |   
| 32 complex | 32 real    | 80 complex    |        4       |   
| 32 complex | 32 complex | 80 complex    |        2       |   
| 32 SPFP    | 32 SPFP    | 32 SPFP       |        8       | 

This table provides some key data points. Specifically, for any combination of data and coefficient you can derive the supported accumulator lane width (“Output” column) and the processing capability in MACs (“Number of MACs”). The latter is the key number for the next step, so make a note of the number of MACs for your data and operand types then continue.

- **Question:** What is the number of MAC operations varies based on the data type? 

To calculate the maximum performance for a given data path, it is necessary to multiply the number of MACs per instruction by the clock frequency of the AI Engine kernel. 

In this example, with 16-bit complex input vectors X(data) and Z(coefficient), the vector processor can achieve 16 MACs per instruction. Using the clock frequency for the slowest speed grade results in:
16 MACs * 1 GHz clock frequency = 16 Giga MAC operations/second.


## Step 3：Select the Number of Lanes

Lanes need some explanation at this point. The lanes are independent processing paths that feed into an accumulator. Lanes are bound together in that they are instanced by a common instruction and share common input vectors but are independent in that their results can be based on different elements of those input vectors.

The two typical math operation for implementing FIR calculations are the mul and mac (multiply and multiply-accumulate). 

For a low-order FIR, which we nominally defined as a FIR where all calculations could be done in a single clock cycle, there is no need to accumulate products across sequential operations since it is done in one. Therefore low-order FIRs will only use the mul intrinsic which will establish how the calculations are set up across all lanes. The situation is similar for higher order FIRs except they require a mul followed by one or more mac intrinsics to sequentially accumulate taps across multiple clock cycles until all taps are calculated. So, for low-order FIR calculations you will need to understand mul. For starters, the intrinsic mul has separate variants for each supported lane configuration as well as for asymmetric and symmetric FIRs.

- **Question:** What's the relationship between lane and samples?

Depending on how the vector is indexed (more on that later) each lane can be a copy of the same FIR working on different data, different coefficients working on the same data, or different coefficients and data.
In each case the total processing capability is split evenly across the number of lanes. The processing capability of each lane (i.e., each FIR) is easily calculated as:

$$
N=\frac{1}{4}\ast MACs ~(for ~ 4 ~ lanes)\\
N=\frac{1}{8}\ast MACs ~(for ~ 8 ~ lanes)\\
N=\frac{1}{16}\ast MACs ~(for ~ 16 ~ lanes)\\
$$

where
- N = achievable number of FIR coefficient multiply/add operations per lane
- MACs = processing capability as noted in table (from the previous step)

Note that N reflects the number of multiply/add operations per lane which is the number of taps for an asymmetric FIR. Symmetric FIR filters with an even number of coefficients will be able to support 2N
taps, while symmetric FIR filters with an odd number of coefficients will be able to support (2N – 1) taps.

For the example FIR, we recorded 16 MACs per clock from the previous step, so the possibilities in terms of the lane options (assuming asymmetry) are:

- 16/4 = 4 taps across 4 lanes

- 16/8 = 8 taps across 2 lanes

## Step 4：Choosing the Proper AIE API for this FIR Application

The AI Engine provides hardware support to accelerate special operations that can be used to accelerate specific application use cases like (but not limited to) signal processing. The Special sliding multiplication classes are listed [here](https://www.xilinx.com/htmldocs/xilinx2022_2/aiengine_api/aie_api/doc/group__group__mul__special.html).

**Taking Advantage of Symmetry**

It was previously noted that the coefficients for the example filter demonstrated symmetry. That means that we have the option of using one of the mul_sym or mul_sym_ct API to implement multipliers with pre-adders, thereby increasing the number of multiply-add operations supported for the selected data and coefficient types. That increase can potentially be applied to:

1. keep the same number of lanes while doubling the number of coefficients (or doubling minus one for an odd number of symmetric coefficients)
2. keep the same number of coefficients while increasing the number of lanes
3. increasing bit width

The derivatives of aie::sliding_mul_sym_ops represent specific behaviors:

``````
sliding_mul_sym_x_ops is similar to sliding_mul_sym_ops, but DataStepY is always 1.
sliding_mul_sym_y_ops is similar to sliding_mul_sym_ops, but DataStepX is always 1.
sliding_mul_sym_xy_ops is similar to sliding_mul_sym_ops, but DataStepX is equal to DataStepY
``````

**Question:** Which AIE API is the best suitable for this FIR application?

As the coefficients are even in number and symmetric, the [sliding_mul_sym_xy_ops](https://www.xilinx.com/htmldocs/xilinx2022_2/aiengine_api/aie_api/doc/group__group__mul__special.html#ga6182f5f9df60d7e1c0fa8deb6b8e3854) will be best suitable for this application.

using mulop = aie::sliding_mul_sym_xy_ops<Lanes, Points, CoeffStep, DataStepXY,int16,int16>; 

The Special sliding multiplication classes support accumulation with different precision such as sliding_mul with 48b accumulation, sliding_mul with 80b accumulation, and sliding_mul with floating-point accumulation. The supported parameters for sliding_mul with 48b accumulation are listed here. 
Here, the consistency between the lanes supported by the AIE sliding multiplication classes and the lanes calculated through computational power in the previous section is validated.

| Types (coeff x data) | Lanes | CoeffStep | DataStepX |  DataStepY  |        coeff_start       | data_start |
|:--------------------:|:-----:|:---------:|:---------:|:-----------:|:------------------------:|:----------:|
| <strong>16b x c16b  <strong>   | <strong>4,8 <strong>  |<strong> 1,2,3,4<strong>   |<strong> 1,2,3,4  <strong> |<strong> 1,2,3,4 1,2<strong> | <strong>Unsigned smaller than 16<strong> | <strong>Signed <strong>    |

## Step 5：Set the Sliding Vector Buffers and Parameterize the Sliding Multiplication API

The most critical aspect that needs to be considered in this step, and also the most crucial aspect in designing a filter using AIE, is how to balance the number of accumulations performed in each loop, determining and maximizing the utilization of data in the sliding buffer while taking into account the known computational power and local register size. This will enable achieving the highest level of parallelism and data utilization effectively.

### 1. A quick look about the AIE vector registers and indexing

The AIE operates on vectors stored in vector registers and delivers results in a vector accumulator. The FIR data and coefficients must thus be delivered in vector form. Vectors are formed by packing C data types together into wider words. The native width for AIE vectors is 128 bits but there are also 256, 512, and 1024 bits vector registers; plus the accumulator can be 320, 384, 640, or 768 bits wide. The naming convention for vectors is as [follows](https://docs.xilinx.com/r/en-US/ug1079-ai-engine-kernel-coding/AI-Engine-Data-Types): 

``````
aie::vector<int16, 32> v;
aie::vector<int32, 16> v2;
aie::vector<cint16, 16> v3;
 
v2 = v.cast_to<int32>(v);
 
v3 = aie::vector_cast<cint16>(v);
``````

Accumulator registers are used to store the results of the vector data path. The accumulation registers are 384 bits wide, which can be viewed as eight vector lanes of 48 bits each.

The idea is to have 32-bit multiplication results and accumulate over those results without bit overflow. The accumulator registers are prefixed with the letters AM. Two AM registers are bundled to form a 768-bit register, prefixed with BM.

In order to fully unlock the powerful vector processing capability of the AI Engine, you must first understand vector buffer data types and the multiplier operators that access them. Vector buffers are long vector data types that can be allocated across multiple vector registers within the A, B, C, and D register units of the AI Engine processor. In memory, they are always allocated on a 128-bit aligned address. A vector contains multiple lanes, each one being an element of the base type of the vector. The vector register table can be found [here](https://docs.xilinx.com/r/en-US/am009-versal-ai-engine/Register-Files).

These buffers (v) are updated via the buffer insert functions (v.insert(N,u)) where u is a 128-bit or 256-bit vector. Once vector data is loaded into multiple lanes of a vector buffer, multiplier operators can be used to perform a variety of multiply-add operations on several lanes simultaneously. The table here summarizes the different vector buffers, how much memory each of them have, and the different configurations they can have as defined by the supported vector data types. 

|     Vector Buffer    |     Total Memory    |     Example – Vector Configuration     |
|:--------------------:|:-------------------:|:--------------------------------------:|
|       XA=WR0+WR1     |         512b        |            vector<cint16,16>           |
|       XB=WR2+WR3     |         512b        |             vector<int16,32>           |
|       XC=WC0+WC1     |         512b        |             vector<cint32,8>           |
|       XD=WD0+WD1     |         512b        |             vector<int8,64>            |
|        YA=XA+XB      |         1024b       |             vector<cint16,32>          |
|        YD=XB+XD      |         1024b       |            vector<cfloat,16>           |


### 2. Select the most suitable size of sliding buffer vector and the optimal number of parallel lanes based on the data type

In this lab, consider an example of a 16-tap filter with 16-bit complex data and 16-bit coefficients. If the 4 lane parallelism is used, it can be clearly to see in the figure below that at least 19 samples are required. 

![loop1](./image/loop1.png)

- **Question 1:** How to set the sliding vector buffer size according to vector data type and register file? 

From the tables above, showing different sizes of vector buffers, we can see that to store 19 samples of data required for parallel computation of every 4 lanes, a vector of length 1024 bits with the vector data type <cint16,32> is needed.

- **Question 2:** Is the vector register being fully utilized? If not, how can we adjust the lanes of each loop and the number of accumulations per operation?

The vector register is not fully utilized. Expanding the parallelism of each loop from 4 lanes to 8 lanes can increase the utilization of the vector register.

![loop2](./image/loop2.png)

### 3. Parameterize the sliding multiplication API

Using the aie::sliding_mul_sym_xy_ops<> function, the data types are defined for the coefficient as int16, the data element as cint16, and the accumulator as acc48.

The final sliding multiplication API parameters are determined based on the FIR taps, lanes, and data type as follows:

```c++
	constexpr unsigned Lanes=4, Points=16, CoeffStep=1, DataStepXY=1;
	using mulop = aie::sliding_mul_sym_xy_ops<Lanes, Points, CoeffStep, DataStepXY,int16,cint16>;
```

There are two functions that are defined with this operator as shown here:

		mul_sym() performs the symmetric multiplication pattern.
		mac_sym() performs the symmetric multiply-add pattern.

- **Question 3:** In theory, based on the computational capability of the AIE, how many clock cycles are needed to obtain one filter output?

Two cycle. 

The light blue data in the above graph represents four accumulations per lane in the first cycle, while the dark blue data represents four accumulations per lane in the second cycle. Eventually, after eight accumulations, the result of a symmetric filter with 16 coefficients is obtained.		


In the generated assembly code below, there are two vector instructions: VMUL and VMAC. This operation takes two cycles. As you can see, 16 operations can be performed in one cycle. 

		VMUL.48 aml0, yd.c16, r10, c0, r1, wc0.c16, #0, c1, c2
		VMAC.48 aml0, yd.c16, r8, c0, r1, wc0.c16, #2, c1, c2

VMUL is a vector multiplication that performs the first 16 int16 x cint16 operations

VMAC is a vector multiplication accumulation that performs the 16 remaining operations.

The 16 operations in one cycle include the pre-addition, multiplication, and post-addition (including the accumulation for VMAC).

## Step 6: Choose the Proper Interface Type for Data Input and Output

Since the kernel consumes and produces data there needs to be a mechanism to move data vectors. This is handled by calls to the window and streaming data API. The API handles the details of synchronizing data, allowing the kernel developer to focus on the data. The actual transport mechanism can be:

1. AXI4 streams through the streaming interconnects and switches in the AIE array,
2. windowed through the AIE memory blocks and DMA hardware
3. streaming across the dedicated cascade interface from/to an adjacent AIE
4. Packetized streams to share a common path

### 1. Interface Considerations

- Asymmetric FIRs

Asymmetric FIRs are perfect fit for utilizing streams. Good overview is available [here](https://github.com/Xilinx/Vitis-Tutorials/blob/master/AI_Engine_Development/AIE/Design_Tutorials/02-super_sampling_rate_fir/SingleKernel/README.md). 

- Symmetric FIRs, including half-bands FIRs

The symmetric FIR filters, particularly those with half-bands, do not scale well to harness the benefits of a streaming interface.
Symmetric FIRs are characterized by mirrored coefficients, which allow data pre-adding and multiplying result by the common coefficients.

As a result, number of MAcs is reduced by up to 50% (odd/even). However, the pre-add stage requires access to data samples from 2 non-contiguous areas in sample memory. For this example design set our Symmetric FIR up with window buffer inputs/outputs

### 2. Interface Declaration

The type declared for the kernel’s inputs and outputs is what establishes the type of interface. For this example design, set our Symmetric FIR up with window inputs/outputs as follows:

```c++
        void fir_16taps_symm(input_window_int16* restrict in, output_window_int16* restrict out)
```

Likewise, we could have declared the asymmetric FIR inputs/outputs as AXI4 streams using

```c++
        void fir_16taps_symm(input_stream_int16* restrict in, output_stream_int16* restrict out).
```

## Step 7: Set the Vector Read and Write API While Tracking the Current Position of Pointer

These API calls for windowed kernel reads and writes track the current position within the vectorized data using a pointer. The API handles pointer initialization, incrementing, and decrementing as data moves. The kernel developer does not need to be concerned with the actual value of the pointer, just the relative motion. 

AIE API has various interface types read input data and write output data

1. Stream interface

readincr_v , writeincr
TLAST value can be handled

2. Window interface

window_read_v , window_readincr_v , window_readdecr_v

window_write , window_writeincr

- **Question 4:** Whether the pointers should be decreased before the next loop begins and why?

The following code snippet shows that in a loop, a total of 24 samples of data are loaded, and the pointer increases by 16 and then decreases by 8 which matches the number of lanes in each loop.

```c++
		sbuff.insert(0, window_readincr_v<8>(cb_input)); 	
		sbuff.insert(1, window_readincr_v<8>(cb_input));	
		sbuff.insert(2, window_read_v<8>(cb_input));	
		
		window_decr_v<8>(cb_input,1);
```

The following figure demonstrate this in action: Input to the kernel is loaded into sbuff as an aie::vector<cint16, 32> vector, and totally 24 samples should be loaded within each single loop. Note that while the vector buffer register should be either 128-bit or 256-bit wide to match the vector load capability of the AIE processor. The window_readincr_v<8>(input) is selected to be align with the 128bit boundary.


![loop_pointer](./image/loops_pointer.png)


## Step 8: Putting it All Together -- Kernel Code Analysis

This code appears to be an implementation of a Finite Impulse Response (FIR) filter with 16 taps and symmetric coefficients. The code takes an input window of int16 data (cb_input) and produces an output window (cb_output) after applying the FIR filtering operation.

Here is an explanation of the code step by step:

The fir_16taps_symm function is declared with two input parameters - cb_input, which is a pointer to an input_window<int16> structure, and cb_output, which is a pointer to an output_window<int16> structure.

**shift** is a constant representing the number of bits to shift the final result for normalization or rounding purposes. **coeff** is a vector of int16 containing the FIR filter coefficients. These coefficients are loaded from memory (assumed to be stored in taps) into the coeff vector.

**sbuff** is defined as a vector of 32 int16 elements. It is used as a sliding buffer to store a portion of the input window during the filtering operation. LSIZE is calculated as samples / 4. 
The code assumes that the number of samples is an integer power of 2 and greater than or equal to 64. This is because each iteration processes 2 sets of 8 samples, totaling 16 samples.

The code enters a loop that iterates over LSIZE times, processing 2 sets of 8 samples per iteration.Inside the loop, the sbuff vector is filled with new data from the cb_input window using the window_readincr_v<8> and window_read_v<8> functions. window_readincr_v<8> reads 8 samples and increments the window pointer by 8, while window_read_v<8> reads 8 samples without incrementing the window pointer.

The code defines a few constants related to the FIR filtering operation, including the number of lanes (Lanes), the number of data points (Points), the coefficient step size (CoeffStep), and the data step size (DataStepXY). The aie::sliding_mul_sym_xy_ops class is used for sliding multiplication of the coefficients with the data in sbuff. This class performs the symmetric FIR filtering operation efficiently using vector operations. The mulop::mul_sym function is called twice to perform the FIR filtering operation for two sets of 8 samples. 

The filtered output is written to the cb_output window using the window_writeincr function and is also shifted by the value in the shift variable before being written. 
The cb_input window pointer is decremented by 8, preparing it for the next iteration to process the next set of 8 samples. The loop continues until all sets of 8 samples in cb_input are processed.

```c++
#include <adf.h>
#include "system_settings.h"
#include "aie_kernels.h"

alignas(32) int16_t taps[FIR_LEN] = {-29, -115, -122, 268, 1379, 3209, 5226, 6566, 6566, 5226, 3209, 1379,  268, -122, -115, -29};//2^15

void fir_16taps_symm // _single_buf_array_indexing
(
		input_window<cint16> *cb_input,
		output_window<cint16> *cb_output
)
{
	const unsigned samples  = window_size(cb_output);
	const int shift         = SRS_SHIFT ;

	const aie::vector<int16,FIR_LEN> coeff = aie::load_v<FIR_LEN>((int16 *)taps) ;
	aie::vector<cint16,32> sbuff;

	const unsigned LSIZE = (samples / 4); // assuming # of samples is integer power of 2 and >= 64

	for ( unsigned i=0; i<LSIZE; i+=2)
	{
		sbuff.insert(0, window_readincr_v<8>(cb_input)); 	// 00++|04++|____|____    ____|____|____|____
		sbuff.insert(1, window_readincr_v<8>(cb_input));	// 00..|04..|08++|12++    ____|____|____|____
		sbuff.insert(2, window_read_v<8>(cb_input)); 		// 00..|04..|08..|12..    16++|20++|____|____

		constexpr unsigned Lanes=4, Points=16, CoeffStep=1, DataStepXY=1;
		using mulop = aie::sliding_mul_sym_xy_ops<Lanes, Points, CoeffStep, DataStepXY,int16,cint16>;

		auto acc = mulop::mul_sym(coeff,0,sbuff,0); // o0..3  =f(c0..7, d0..10,  d8..18)
	    window_writeincr(cb_output, acc.to_vector<cint16>(shift));

		acc = mulop::mul_sym(coeff,0,sbuff,4); // o4..7  =f(c0..7, d4..14,  d12..22)
	    window_writeincr(cb_output, acc.to_vector<cint16>(shift));

		window_decr_v<8>(cb_input,1);
	}
}
```

--------------

 

<center>

 

Copyright&copy; 2023 AMD, Inc

 

SPDX-License-Identifier: MIT

 

</center>